In [1]:
# 2023-Apr-26
# Jupyter notebook for correcting channel drift, since 2002, in SI spectra.

In [2]:
import numpy as np
from numpy.random import default_rng
from numpy.polynomial import Polynomial as P
# from scipy import special
import scipy as sci
import pandas as pd

from ograyspy_class import select_spectrum_from_folder_list
from spec_class import Spec

In [3]:
import plotly.graph_objects as go

In [4]:
# a spectrum pattern to find and test:

In [5]:

# 2022-out-7: Excelente espectro para testes, tenho usado ultimamente:
a_pattern = 'SI2018/SI11318.Chn'

In [6]:
from ograyspy_class import Ograyspy

In [7]:
ogra = Ograyspy(batch_mode=False, folder_to_find='Genie_Transfer/Si')
print('Objeto ogra: Ograyspy criado.')
print(f'ogra.info_plat: {ogra.info_plat}')
print(f'ogra.info_mach: {ogra.info_mach}')
print(f'ogra.info_syst: {ogra.info_syst}')
print(f'ogra.info_node: {ogra.info_node}')
print(f'ogra.home_path: {ogra.home_path}')

Found folder name:  /home/marcelofm/Nextcloud/Genie_Transfer/Si
Partes:  ('/', 'home', 'marcelofm', 'Nextcloud', 'Genie_Transfer', 'Si')
No. spec files:  1009
Objeto ogra: Ograyspy criado.
ogra.info_plat: Linux-6.2.0-20-generic-x86_64-with-glibc2.37
ogra.info_mach: x86_64
ogra.info_syst: Linux
ogra.info_node: mmaduar-net13
ogra.home_path: /home/marcelofm


In [8]:
# Select a machine/system to read from:
spectra_list_df = pd.read_pickle(ogra.pkl_folder_files)
spectra_list_df

,info_plat,info_mach,info_syst,info_node,home_path,spectra_path,n_files,a_spec_ind,a_spec_name,reduced_f_name,files_list,reduced_names_files_list,spectra_pattern_names,pkl_folder_files
0,Linux-6.2.0-20-generic-x86_64-with-glibc2.37,x86_64,Linux,mmaduar-net13,/home/marcelofm,/home/marcelofm/Nextcloud/Genie_Transfer/Si,1009,0,,,[/home/marcelofm/Nextcloud/Genie_Transfer/Si/S...,"[Si2006/SI12306.Chn, Si2006/SI02606.Chn, Si200...","[**/*.[Cc][Hh][Nn], **/*.[Ii][Ee][Cc]]",Linuxmmaduar-net13.pkl


In [9]:
reduc_nms = spectra_list_df.reduced_names_files_list[0]
fil_lst = spectra_list_df.files_list[0]
spc_pth = spectra_list_df.spectra_path[0]

In [10]:
a_spec_name, reduced_f_name = select_spectrum_from_folder_list(reduc_nms, fil_lst, spc_pth, a_pattern)

Existing:
Achou! indice=302, nomearq = SI2018/SI11318.Chn
name:  /home/marcelofm/Nextcloud/Genie_Transfer/Si/SI2018/SI11318.Chn
Final choices:
spectra_path: /home/marcelofm/Nextcloud/Genie_Transfer/Si
a_spec_name: /home/marcelofm/Nextcloud/Genie_Transfer/Si/SI2018/SI11318.Chn
reduced_f_name: SI2018/SI11318.Chn


In [11]:
a_spec_name, reduced_f_name

(PosixPath('/home/marcelofm/Nextcloud/Genie_Transfer/Si/SI2018/SI11318.Chn'),
 'SI2018/SI11318.Chn')

In [12]:
a_spec = Spec(a_spec_name, reduced_f_name)

In [13]:
x_nz = a_spec.gross_spec_ser_an.chans_nzero
y_nz = a_spec.gross_spec_ser_an.counts_nzero

In [14]:
gross_ser = a_spec.gross_spec_ser_an

In [15]:
chans = gross_ser.x_s
counts = gross_ser.y_s
chans_nzero = gross_ser.chans_nzero
counts_nzero = gross_ser.counts_nzero
unc_y = gross_ser.unc_y


In [16]:
fig2 = go.Figure(data=go.Scattergl(
    x=x_nz,
    y=y_nz,
    mode='markers'
))


In [17]:
fig2.add_trace(
    go.Scatter(x=chans_nzero,
               y=counts_nzero,
               error_y=dict(
                   color='orange',width=3.0,
                   type='data', # value of error bar given in data coordinates
                   array=unc_y,
                   visible=True),
               name="Counts & uncertaintes",
               line=dict(color='orange',width=0.3)));



In [18]:
fig2.update_layout(
    width=1800,
    height=900,
    margin=dict(l=10, r=10, t=30, b=10),
    paper_bgcolor="LightSteelBlue",
)
fig2.write_html('fig2.html')


In [19]:
# Set title and scale type
fig2.update_layout(title_text="Gamma-ray spectrum excerpt")
fig2.update_yaxes(type="log");
fig2.write_html('fig2.html', auto_open=True)


In [20]:
# Agora, o grande loop para comparar o espectro escolhido
# com todos os outros:

In [21]:
for i_sp, j_sp in zip(fil_lst, reduc_nms):
    print(j_sp)
    a_spec = Spec(a_spec_name, reduced_f_name)
    

Si2006/SI12306.Chn
Si2006/SI02606.Chn
Si2006/SI16206.Chn
Si2006/si139.140.Chn
Si2006/si15906.Chn
Si2006/SI01506.Chn
Si2006/SI12506.Chn
Si2006/si00406.Chn
Si2006/si05406.Chn
Si2006/si07206.Chn
Si2006/si16106.Chn
Si2006/si07706r.Chn
Si2006/si12906.Chn
Si2006/si02406.Chn
Si2006/SI10906.Chn
Si2006/SI13106.Chn
Si2006/si12606.Chn
Si2006/si18806.Chn
Si2006/SI12706.Chn
Si2006/si02306.Chn
Si2006/SI06706.Chn
Si2006/SI01906.Chn
Si2006/SI07106.Chn
Si2006/si11306.Chn
Si2006/SI07606.Chn
Si2006/si1503.Chn
Si2006/SI9106.Chn
Si2006/SI7006.Chn
Si2006/si04906r.Chn
Si2006/si09506.Chn
Si2006/SI03106.Chn
Si2006/SI07006.Chn
Si2006/SI16806.Chn
Si2006/SI07906.Chn
Si2006/SI15006.Chn
Si2006/si10406.Chn
Si2006/SI00706.Chn
Si2006/si14606.Chn
Si2006/si129-130-137-13806.Chn
Si2006/si04906.Chn
Si2006/si17906.Chn
Si2006/si17806.Chn
Si2006/SI17506.Chn
Si2006/si06306.Chn
Si2006/SI16006.Chn
Si2006/si05306.Chn
Si2006/si11706.Chn
Si2006/SI05506.Chn
Si2006/SI15106A.Chn
Si2006/si09906.Chn
Si2006/si15006r.Chn
Si2006/si00506.C

si2009/filtro2.Chn
si2009/SI03009.Chn
SI2015/SI04715.Chn
SI2015/SI07015.Chn
SI2015/Regeneracao 1.Chn
SI2015/lavagem final 3.Chn
SI2015/SI09515.Chn
SI2015/SI12015.Chn
SI2015/SI04315.Chn
SI2015/SI14915.Chn
SI2015/SI11915.Chn
SI2015/SI11115.Chn
SI2015/SI3115.Chn
SI2015/Regeneracao 2.Chn
SI2015/Lavagem 3.Chn
SI2015/si01115.Chn
SI2015/SI09015.CHN
SI2015/SI02315.Chn
SI2015/SI14215.Chn
SI2015/SI08815.Chn
SI2015/SI06315.Chn
SI2015/SI09115.Chn
SI2015/SI15015.Chn
SI2015/SI03915-contA.Chn
SI2015/SI11815.Chn
SI2015/SI01515.Chn
SI2015/Lavagem 2.Chn
SI2015/SI04115.Chn
SI2015/SI05115.Chn
SI2015/SI04015.Chn
SI2015/lavagem final 2.Chn
SI2015/SI08415.Chn
SI2015/SI04215.Chn
SI2015/si01015.Chn
SI2015/SI06215.Chn
SI2015/SI02615.Chn
SI2015/SI01915.Chn
SI2015/SI10215.Chn
SI2015/Lavagem final 1.Chn
SI2015/SI12415.Chn
SI2015/SI08915.Chn
SI2015/Regeneracao 3.Chn
SI2015/SI03215.Chn
SI2015/Lavagem 1.Chn
SI2015/SI03915_RECONTAGEM.Chn
SI2015/SI08315.Chn
SI2015/SI10715.Chn
SI2015/SI3015.Chn
si2014/SI12814.Chn
si2014

Si2005/si01805.CHN
Si2005/si07505.CHN
Si2005/si08905.chn
Si2005/si062052.chn
Si2005/si14105.Chn
Si2005/si0172.Chn
Si2005/si1605.chn
Si2005/si01105.chn
Si2005/SI21605.Chn
Si2005/si3205.CHN
Si2005/SI20005.Chn
Si2005/SI20705.Chn
Si2005/SI0705.chn
Si2005/si20605.Chn
Si2005/si5405.chn
Si2005/si19605.Chn
Si2005/si9905.chn
Si2005/SI22105R.Chn
Si2005/SI17805a.Chn
Si2005/si21105.Chn
Si2005/SI11005.chn
Si2005/si19705.Chn
Si2005/SI20405.Chn
Si2005/si20905.Chn
Si2005/si4005.CHN
Si2005/si13305.chn
Si2005/si12905.CHN
Si2005/si21005.Chn
Si2005/si19305.Chn
Si2005/si1305.CHN
Si2005/si10105.chn
Si2005/si11105.CHN
Si2005/si10205.chn
Si2005/SI10605.CHN
Si2005/si21505.Chn
Si2005/SI13205.CHN
Si2005/si19405c.Chn
Si2005/si06505.chn
Si2005/si15305.Chn
Si2005/SI6905.CHN
Si2005/si3605.chn
Si2005/si20505.Chn
Si2005/si20805.Chn
Si2005/SI08105.CHN
Si2005/si19505.Chn
Si2005/si06605.chn
Si2005/SI13705.CHN
Si2005/si04405.chn
Si2005/si12005.chn
Si2005/SI22805.Chn
Si2005/SI08005.CHN
Si2005/SI08205.CHN
Si2005/si19405a.Ch

In [30]:
i_n = 0
for i_sp, j_sp in zip(fil_lst, reduc_nms):
    print(i_n)
    a_spec = Spec(a_spec_name, reduced_f_name)
    i_n += 1
    y_s = gross_ser.y_s
    # y_s = np.where(y_s > 0, np.log(y_s), 0)

    # convo = sci.signal.correlate(y_s,y_s)
    # convo = sci.signal.convolve(counts, counts)
    # convo = sci.signal.fftconvolve(y_s, y_s)
    # convo = sci.signal.fftconvolve(y_s, np.roll(y_s,1500))
    # shifted_y_s = np.roll(y_s,1319)
    # convo = sci.spatial.distance.euclidean(y_s, shifted_y_s)
    # convo = sci.signal.detrend(y_s)
    # convo = sci.signal.correlate(y_s, shifted_y_s)


    # af = sci.fft.fft(y_s)
    # bf = sci.fft.fft(shifted_y_s)
    # c = sci.fft.ifft(af * np.conj(bf))

    # time_shift = np.argmax(abs(c))


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [31]:
i_n

1009

In [ ]:
# 2023-Apr-26 PAREI AQUI
aesgre graesrg egr

In [ ]:
y_s = gross_ser.y_s
# y_s = np.where(y_s > 0, np.log(y_s), 0)

convo = sci.signal.correlate(y_s,y_s)
# convo = sci.signal.convolve(counts, counts)
convo = sci.signal.fftconvolve(y_s, y_s)
# convo = sci.signal.fftconvolve(y_s, np.roll(y_s,1500))
shifted_y_s = np.roll(y_s,1319)
# convo = sci.spatial.distance.euclidean(y_s, shifted_y_s)
# convo = sci.signal.detrend(y_s)
convo = sci.signal.correlate(y_s, shifted_y_s)


af = sci.fft.fft(y_s)
bf = sci.fft.fft(shifted_y_s)
c = sci.fft.ifft(af * np.conj(bf))

time_shift = np.argmax(abs(c))




np.argmax(convo)

In [ ]:
print(time_shift)

In [ ]:
a = y_s
b = shifted_y_s

In [ ]:
a_to_b = np.argmax(sci.signal.correlate(a,b))
b_to_a = np.argmax(sci.signal.correlate(b,a))

In [ ]:
a_to_b

In [ ]:
b_to_a

In [ ]:
convo = c

In [ ]:
c

In [ ]:
figco = go.Figure(data=go.Scattergl(
    x=chans,
    y=convo,
    mode='markers'
))

figco.update_layout(
    width=900,
    height=500,
    margin=dict(l=10, r=10, t=30, b=10),
    paper_bgcolor="LightSteelBlue",
)
figco.write_html('figco.html')
figco.show()

In [ ]:
np.where(y_s > 0, np.log(y_s), 0)

In [ ]:
fig2.add_trace(
    go.Scatter(x=chans,
               y=eval_y,
               name="eval_y",
               line=dict(color='blue',width=0.5)));

In [ ]:
figw1b.add_trace(
    go.Scatter(x=chans,
               y=eval_bl,
               name='eval_bl',
               line=dict(color='green',width=0.5)));

In [ ]:
figw1b.add_trace(
    go.Scatter(x=chans,
               y=fin_bl,
               name='fin_bl',
               line=dict(color='red',width=0.5)));

In [ ]:
figw1b.add_trace(
    go.Scatter(x=peaks_1,
               y=counts[peaks_1],
               mode="markers",
               name="peaks",
               marker=dict(color='cyan',
                           symbol='cross',
                           size=20,
                           opacity=0.5,
                           line=dict(color='red', width=2.0)
                          )
              ));

In [ ]:
figw1b.add_trace(
    go.Scatter(x=peaks_net,
               y=counts[peaks_net],
               mode="markers",
               name="net peaks",
               marker=dict(color='red',
                           symbol='circle',
                           size=20,
                           opacity=0.4,
                           line=dict(color='green', width=2.0)
                          )
              ));


# Set title and scale type
figw1b.update_layout(title_text="Gamma-ray spectrum excerpt")
figw1b.update_yaxes(type="log");

# Melhor não abrir inline, que seria simplesmente entrar com o nome do objeto:
# figw2
# Além de ficar com visualização incômoda, não funciona no JupyterLab; dá o erro:
# Error displaying widget: model not found.

In [ ]:
figw1b.add_trace(
    go.Scatter(x=inis,
               y=eval_y[inis],
               mode="markers",
               name="inis",
               marker=dict(color='yellow',
                           symbol='pentagon',
                           size=10,
                           opacity=1.0,
                           line=dict(color='green', width=2.0)
                          )
              ));